# Package that we need

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import os
from sklearn import svm
from scipy.stats import multivariate_normal


# you can choose one of the following package for image reading/processing

import cv2
import PIL


## 1 Support Vector Machine

<style>
.blue{
    color: skyblue;
}
.bold{
    font-weight: bold;
}
</style>

In the training procedure of SVM, we need to optimize with respect to the Lagrange multipliers
$a = \{ a_n \}$. 

Here, we use the <span class="blue">sequential minimal optimization</span> to solve the problem. 

For details, you can refer to the paper [Platt, John. “Sequential minimal optimization: 

A fast algorithm for training support vector machines”, 1998]. The classifier is written by

$$
    y(\textbf{x}) = \sum_{n=1}^N a_n t_n k(\textbf{x}, \textbf{x}_n) = \textbf{w}^T \textbf{x} + b
$$
$$
    \textbf{w} = \sum_{n=1}^N \alpha_n t_n \phi(\textbf{x}_n)
$$
$$
    b = \frac{1}{N_{\mathcal{M}}} \sum_{n \in \mathcal{M}} \left( t_n - \sum_{m \in \mathcal{S}} a_m t_m k(\textbf{x}_n, \textbf{x}_m) \right)
$$

where $\mathcal{M}$ denotes the set of indices of data points having $0 \lt a_n \lt C$.

### 1.1

It is popular to use principal component analysis (PCA) to reduce the dimension of images to d = 2. 

Please implement it by yourself instead of using the method from sklearn.

In [ ]:
data = pd.read_csv("./x_train.csv",header= None)/255
label = pd.read_csv("./t_train.csv",header= None)

**PCA**

$\textbf{C}$ is covariance matrix of $\textbf{X}$, $\textbf{W}$ is the eigenvector matrix of $\textbf{C}$, $\Lambda$ is the eigenvalue matrix of $\textbf{C}$.

$\textbf{X}$ is the data matrix, $\textbf{X}_k$ is the data matrix after dimension reduction, and k in this task will be 2.

$$
    \textbf{C} = \frac{\textbf{X}^T \textbf{X}}{n - 1}
$$
$$
    \textbf{C} = \textbf{W} \Lambda \textbf{W}^{-1}
$$
$$
    \textbf{X}_k = \textbf{X} \textbf{W}_k
$$

**Using SVD to solve PCA**

$\Sigma$ is the singular value matrix of $\textbf{X}$, $\textbf{U}$ is the left singular vector matrix of $\textbf{X}$, $\textbf{V}$ is the right singular vector matrix of $\textbf{X}$.

$$
    \textbf{C} = \frac{\textbf{X}^T \textbf{X}}{n - 1} 
$$
$$
    \textbf{C} = \frac{\textbf{V}\Sigma\textbf{U}^T\textbf{U}\Sigma\textbf{V}^T}{n - 1}
$$
$$
    \textbf{C} = \textbf{V} \frac{\Sigma^2}{n - 1}\textbf{V}^T
$$
$$
    \textbf{C} = \textbf{V} \frac{\Sigma^2}{n - 1}\textbf{V}^{-1} \ \ (\rm{because} \ \textbf{V} \ \rm{is \ unitary})
$$

We can found that covariance matrix $\textbf{C}$ is the same as the eigenvalue matrix $\Lambda$ of $\textbf{V}^{-1}$.

$$
    \Lambda = \frac{\Sigma^2}{n - 1}
$$

In [ ]:
data_new = data.values - np.mean(data.values, axis=0) # centralize data
np.allclose(data_new.mean(axis=0), np.zeros(data_new.shape[1])) # check if centralization is correct
C = (data_new.T @ data_new) / (data_new.shape[0] - 1) # covariance matrix
eig_vals, eig_vecs = np.linalg.eig(C) # eigenvalues and eigenvectors
eig_vals, eig_vecs = eig_vals.real, eig_vecs.real # convert to real numbers
# sort eigenvalues in ascending order   
eig_vals_sorted = np.argsort(eig_vals)[::1][-2:] # sort eigenvalues in descending order and take the first two
data_pca = data_new @ eig_vecs[:,eig_vals_sorted] # PCA

In [ ]:
# plot 2D data with label
plt.figure(figsize=(10,10))
plt.scatter(data_pca[:,0],data_pca[:,1],c=label.values)
plt.show()

#### 1.2

Describe the difference between two decision approaches (one-versus-the-rest and one-
versus-one). 

Decide which one you want to choose and explain why you choose this approach.

<style>
.blue{
    color: skyblue;
}
.red{
    color: red;
}
.bold{
    font-weight: bold;
}
</style>

**One-versus-the-rest**

First, we train a binary classifier <span class="red">for each class</span>. 

Only one class is positive and the rest are negative.

Then, we use the classifier to predict the class of the test data. 

Finally, we choose the class with the highest score.

**One-versus-one**

We train a binary classifier <span class="red">for each pair of classes</sapn>. 

Then, we use the classifier to predict the class of the test data. 

The test data is assigned to the class that wins the most duels.

Finally, we choose the class with the highest score.

**Which one to choose**

Support vector machine is a binary classifier.

If we use one-versus-the-rest, we need to train $C$ binary classifiers.

If we use one-versus-one, we need to train $\frac{C(C-1)}{2}$ binary classifiers.

If $C$ is large, <span class = "red">one-versus-one is better</sapn>.

#### 1.3

<style>
.blue{
    color: skyblue;
}
.bold{
    font-weight: bold;
}
</style>

Use the principle values projected to top <span class="blue">two</span> eigenvectors obtained from PCA, and build
a SVM with <span class="blue">linear kernel</span> to do multi-class classification. 

You can decide the upper bound $C$ of $a_n$ by yourself or just use the default value provided by sklearn. 

Then, <span class="blue">plot the corresponding decision boundary</span> and show the <span class="blue">support vectors.</span>

* Linear kernel:
$$
    k(\textbf{x}_i, \textbf{x}_j) = \phi(\textbf{x}_i)^T \phi(\textbf{x}_j) = \textbf{x}_i^T \textbf{x}_j
$$

The sample figures are provided below.

<center>
    <img src = "./image/figure1.png" width = 50%>
</center>

In [ ]:
# https://github.com/scikit-learn/scikit-learn/blob/dc580a8ef/sklearn/svm/_classes.py#L554
# svm with linear kernel and C=1
from sklearn import svm
clf = svm.SVC(kernel='linear', C=1)
clf.fit(data_pca, label.values.ravel())
w = clf.coef_[0]
a = -w[0] / w[1]
xx = np.linspace(-0.5, 0.5)
yy = a * xx - (clf.intercept_[0]) / w[1]
plt.figure(figsize=(10,10))
plt.scatter(data_pca[:,0],data_pca[:,1],c=label.values)
plt.plot(xx, yy, 'k-')
plt.show()

#### Bonus

<style>
.blue{
    color: skyblue;
}
.bold{
    font-weight: bold;
}
</style>

Repeat 3 with <span class="blue">polynomial kernel (degree = 2).</span>

* Polynomial (homogeneous) kernel of degree 2:

$$
    k(\textbf{x}_i, \textbf{x}_j) = \phi(\textbf{x}_i)^T \phi(\textbf{x}_j) = \left( \textbf{x}_i^T \textbf{x}_j\right)^2
$$
$$
    \phi(\textbf{x}) = \left[ \textbf{x}_1^2, \sqrt{2} \textbf{x}_1 \textbf{x}_2, \textbf{x}_2^2 \right]
$$
$$
    \textbf{x} = \left[ x_1, x_2 \right]
$$

## 2 Gaussian Mixture Model

<style>
.blue{
    color: skyblue;
}
.red{
    color: red;
}
.bold{
    font-weight: bold;
}
</style>

In this exercise, you will implement a Gaussian mixture model (GMM) and apply it in image segmentation. 

First, use a $K$-means algorithm to find $K$ central pixels. 

Second, use the expectation maximization (EM) algorithm <span class="blue">(please refer to textbook p.438-p.439)</span> to optimize the parameters of the model. 

The input image is given by <span class="red">hw3.jpg</span>. 

According to the maximum likelihood, you can decide the color $\mu_k$ , $k \in [1, . . . , K]$ of each pixel $x_n$ of output image

#### 2.1

Please build a $K$-means model by minimizing

$$
    J = \sum_{n=1}^N \sum_{k=1}^K \gamma_{nk} \| x_n - \mu_k \|^2
$$

and show the table of the estimated $\{ \mu_k \}^K_{k=1}$.

In [ ]:
np.random.seed(42)
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2)**2))
class KMeans():
    def __init__(self, K=5, max_iters=100, plot_steps=False):
        self.K = K
        self.max_iters = max_iters
        self.plot_steps = plot_steps
        # list of sample indices for each cluster
        self.clusters = [[] for _ in range(self.K)]
        # the centers (mean feature vector) for each cluster
        self.centroids = []
        self.likelihood = []
    def predict(self, X):
        self.X = X
        self.n_samples, self.n_features = X.shape
        
        # initialize 
        random_sample_idxs = np.random.choice(self.n_samples, self.K, replace=False)
        self.centroids = [self.X[idx] for idx in random_sample_idxs]
        # Optimize clusters
        for _ in range(self.max_iters):
            # Assign samples to closest centroids (create clusters)
            self.clusters = self._create_clusters(self.centroids)
            if self.plot_steps:
                self.plot()
            # Calculate new centroids from the clusters
            centroids_old = self.centroids
            self.centroids = self._get_centroids(self.clusters)
            
            # check if clusters have changed
            if self._is_converged(centroids_old, self.centroids):
                break
            if self.plot_steps:
                self.plot()
        # Classify samples as the index of their clusters
        return self._get_cluster_labels(self.clusters)
    def _get_cluster_labels(self, clusters):
        # each sample will get the label of the cluster it was assigned to
        labels = np.empty(self.n_samples)
        for cluster_idx, cluster in enumerate(clusters):
            for sample_index in cluster:
                labels[sample_index] = cluster_idx
        return labels
    def _create_clusters(self, centroids):
        # Assign the samples to the closest centroids to create clusters
        clusters = [[] for _ in range(self.K)]
        for idx, sample in enumerate(self.X):
            centroid_idx = self._closest_centroid(sample, centroids)
            clusters[centroid_idx].append(idx)
        return clusters
    def _closest_centroid(self, sample, centroids):
        # distance of the current sample to each centroid
        distances = [euclidean_distance(sample, point) for point in centroids]
        closest_index = np.argmin(distances)
        return closest_index
    def _get_centroids(self, clusters):
        # assign mean value of clusters to centroids
        centroids = np.zeros((self.K, self.n_features))
        for cluster_idx, cluster in enumerate(clusters):
            cluster_mean = np.mean(self.X[cluster], axis=0)
            centroids[cluster_idx] = cluster_mean
        return centroids
    def _is_converged(self, centroids_old, centroids):
        # distances between each old and new centroids, fol all centroids
        distances = [euclidean_distance(centroids_old[i], centroids[i]) for i in range(self.K)]
        return sum(distances) == 0
    def plot(self):
        fig, ax = plt.subplots(figsize=(12, 8))
        for i, index in enumerate(self.clusters):
            point = self.X[index].T
            ax.scatter(*point)
        for point in self.centroids:
            ax.scatter(*point, marker="x", color='black', linewidth=2)
        plt.show()
    def cent(self):
        return self.centroids

In [ ]:
image = cv2.imread("./hw3.jpg")
pixel_values = image.reshape((-1, 3))
pixel_values = np.float32(pixel_values)
print(pixel_values.shape)

In [ ]:
# # EM algorithm using numpy
# def EM(X, K, max_iter=100):
#     N, D = X.shape
#     # initialize
#     mu = np.random.randn(K, D)
#     sigma = np.random.randn(K, D, D)
#     pi = np.random.randn(K)
#     pi = pi / np.sum(pi)
#     for i in range(K):
#         sigma[i] = np.eye(D)
#     for i in range(max_iter):
#         # E-step
#         gamma = np.zeros((N, K))
#         for k in range(K):
#             gamma[:, k] = pi[k] * multivariate_normal.pdf(X, mean=mu[k], cov=sigma[k])
#         gamma = gamma / np.sum(gamma, axis=1, keepdims=True)
#         # M-step
#         Nk = np.sum(gamma, axis=0)
#         for k in range(K):
#             mu[k] = np.sum(gamma[:, k].reshape(-1, 1) * X, axis=0) / Nk[k]
#             sigma[k] = np.dot((gamma[:, k].reshape(-1, 1) * (X - mu[k])).T, X - mu[k]) / Nk[k]
#         pi = Nk / N
#     return mu, sigma, pi

# # remove nan
# pixel_values = np.nan_to_num(pixel_values)
# pixel_values = pixel_values / 255

# print(pixel_values.shape)

# # postitive definite
# for i in range(3):
#     pixel_values[:, i] = pixel_values[:, i] + 1e-5

# mu, sigma, pi = EM(pixel_values, 3)
# print(mu)
# print(sigma)
# print(pi)

#### 2.2

<style>
.blue{
    color: skyblue;
}
.red{
    color: red;
}
.bold{
    font-weight: bold;
}
</style>

Use $ \mu = \{ \mu_k \}^K_{k=1}$ calculated by the $K$-means model as the means, and calculate the corresponding variances $ \sigma_k^2 $ 

and mixing coefficient $\pi_k$ for the initialization of the GMM $p(x) = \sum_{k=1}^K \pi_k \mathcal{N}(x \ | \ \mu_k, \sigma_k^2)$.

Optimize the model by maximizing the log likelihood function $\log \ p(x \ | \ \pi, \mu, \sigma^2)$ over N pixels through EM algorithm. 

<span class="red">Plot the learning curve for log likelihood of GMM.</span> <span class="blue">(Please terminate EM algorithm when the number of iterations arrives at 100.)</span>

#### 2.3

Repeat steps 1 and 2 for $K = 2, 3, 7$ and $20$. Please show the resulting images of $K$-means model and GMM, respectively. 

Below are some examples.

<center>
    <img src = "./image/figure2.png" width = 50%>
</center>

In [ ]:
k_slot = [2, 3, 7, 20]
for k_idx in k_slot:
    k = KMeans(K=k_idx, max_iters=100)  
    y_pred = k.predict(pixel_values)
    centers = np.uint8(k.cent())
    y_pred = y_pred.astype(int)
    labels = y_pred.flatten()
    segmented_image = centers[labels.flatten()]
    segmented_image = segmented_image.reshape(image.shape)
    plt.title("K = {}".format(k_idx))
    plt.imshow(segmented_image)
    plt.show()

#### 2.4

Make some discussion about what is crucial factor to affect the output image between $K$-means and Gaussian mixture model (GMM), and explain the reason.

#### 2.5

The input image shown below comes from the licence-free dataset for personal and commercial use. 

Image from: https://pickupimage.com/free-photos/Cat-in-the-forest/2333003

<center>
    <img src = "./image/figure3.png" width = 30%>
</center>